In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *

sys.path.insert(0, "../../scripts/methods/")
from my_moscot import moscot_align, moscot_align_ref
import time
from tqdm import tqdm

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

(EE) 
Fatal server error:
(EE) Server is already active for display 99
	If this server is no longer running, remove /tmp/.X99-lock
	and start again.
(EE) 


In [2]:
# Load data
data_folder = "../../data/BARseq/BARseq_Perturbed/"
results_folder = "./results/Moscot/"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
subsample_num = 20000
  
for i in tqdm(range(1, 40)):
    slice1 = ad.read_h5ad(filename=os.path.join(data_folder,"slice_{:0>2d}.h5ad".format(i)))
    slice2 = ad.read_h5ad(filename=os.path.join(data_folder,"slice_{:0>2d}.h5ad".format(i+1)))
    
    # subsample
    subsample1 = np.random.choice(slice1.shape[0], subsample_num, replace=False) if slice1.shape[0] > subsample_num else np.arange(slice1.shape[0])
    subsample2 = np.random.choice(slice2.shape[0], subsample_num, replace=False) if slice2.shape[0] > subsample_num else np.arange(slice2.shape[0])
    slice1 = slice1[subsample1,:]
    slice2 = slice2[subsample2,:]

    spatial_key = 'perturbed_spatial'
    key_added = 'align_spatial'
    align_slices, pis = moscot_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="cuda",
        verbose=False,
        alpha=0.5,
    )    
    # recover true R and t
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm['perturbed_spatial'])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm['perturbed_spatial'], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    results = {"R":R, "t":t, "subsample": [subsample1, subsample2]}
    np.save(os.path.join(results_folder, "slice_{}_{}.npy".format(i,i+1)), results, allow_pickle=True)

  0%|                                                           | 0/39 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  


/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(20000, 20000)].                                        


  3%|█▎                                                 | 1/39 [00:41<26:18, 41.54s/it]

INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  


/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(20000, 20000)].                                        
